# 🔍 Analyse du Data Drift

Ce notebook analyse la dérive des données entre l'entraînement et la production.

## Objectifs
1. Comparer les distributions des features
2. Détecter les drifts statistiques
3. Visualiser les changements
4. Recommandations pour le re-entraînement

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Chargement des données

In [ ]:
# Charger les données de référence (entraînement)
reference_df = pd.read_csv('../data/reference_data.csv')
print(f"📊 Données de référence: {len(reference_df)} échantillons")
print(f"Colonnes: {reference_df.columns.tolist()}")
reference_df.head()

In [ ]:
# Charger les logs de production
production_logs = []
with open('../production_logs.json', 'r') as f:
    for line in f:
        if line.strip():
            production_logs.append(json.loads(line))

production_df = pd.DataFrame(production_logs)
input_df = pd.json_normalize(production_df['input'])

print(f"📊 Données de production: {len(input_df)} échantillons")
print(f"Colonnes: {input_df.columns.tolist()}")
input_df.head()

## 2. Statistiques descriptives

In [ ]:
# Comparaison des statistiques
features = ['age', 'income', 'loan_amount', 'employment_length', 'credit_score']

print("\n📊 STATISTIQUES DESCRIPTIVES\n")
print("=" * 80)

for feature in features:
    if feature in reference_df.columns and feature in input_df.columns:
        print(f"\n{feature.upper()}:")
        print(f"  Référence - Moyenne: {reference_df[feature].mean():.2f}, Std: {reference_df[feature].std():.2f}")
        print(f"  Production - Moyenne: {input_df[feature].mean():.2f}, Std: {input_df[feature].std():.2f}")
        
        # Différence en %
        diff = ((input_df[feature].mean() - reference_df[feature].mean()) / reference_df[feature].mean() * 100)
        print(f"  Différence: {diff:+.2f}%")

## 3. Tests statistiques de drift

In [ ]:
# Test de Kolmogorov-Smirnov pour chaque feature
drift_results = []

print("\n🔍 DÉTECTION DE DRIFT (Test Kolmogorov-Smirnov)\n")
print("=" * 80)

for feature in features:
    if feature in reference_df.columns and feature in input_df.columns:
        # KS test
        ks_stat, p_value = stats.ks_2samp(
            reference_df[feature].dropna(),
            input_df[feature].dropna()
        )
        
        # Interpréter le résultat
        drift_detected = p_value < 0.05
        status = "🔴 DRIFT DÉTECTÉ" if drift_detected else "🟢 OK"
        
        drift_results.append({
            'Feature': feature,
            'KS Statistic': ks_stat,
            'P-Value': p_value,
            'Drift': drift_detected,
            'Status': status
        })
        
        print(f"\n{feature}:")
        print(f"  KS Statistic: {ks_stat:.4f}")
        print(f"  P-Value: {p_value:.4f}")
        print(f"  Status: {status}")

# Créer un DataFrame des résultats
drift_df = pd.DataFrame(drift_results)
drift_df

## 4. Visualisations des distributions

In [ ]:
# Comparaison des distributions
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, feature in enumerate(features):
    if feature in reference_df.columns and feature in input_df.columns:
        ax = axes[idx]
        
        # Histogrammes
        ax.hist(reference_df[feature].dropna(), bins=30, alpha=0.5, label='Référence', color='blue')
        ax.hist(input_df[feature].dropna(), bins=30, alpha=0.5, label='Production', color='red')
        
        ax.set_title(f'Distribution - {feature}', fontsize=12, fontweight='bold')
        ax.set_xlabel(feature)
        ax.set_ylabel('Fréquence')
        ax.legend()
        ax.grid(True, alpha=0.3)

# Supprimer le dernier subplot s'il n'est pas utilisé
if len(features) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.savefig('../notebooks/drift_analysis_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Graphique sauvegardé: drift_analysis_distributions.png")

## 5. Box plots comparatifs

In [ ]:
# Créer des box plots
fig, axes = plt.subplots(1, len(features), figsize=(20, 4))

for idx, feature in enumerate(features):
    if feature in reference_df.columns and feature in input_df.columns:
        # Préparer les données
        data_to_plot = [
            reference_df[feature].dropna(),
            input_df[feature].dropna()
        ]
        
        axes[idx].boxplot(data_to_plot, labels=['Référence', 'Production'])
        axes[idx].set_title(feature, fontweight='bold')
        axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../notebooks/drift_analysis_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Graphique sauvegardé: drift_analysis_boxplots.png")

## 6. Analyse approfondie avec Evidently

In [ ]:
# Utiliser Evidently pour une analyse plus poussée
try:
    from evidently import ColumnMapping
    from evidently.report import Report
    from evidently.metric_preset import DataDriftPreset, DataQualityPreset
    
    # Préparer les données
    reference_data = reference_df[features]
    production_data = input_df[features]
    
    # Créer le rapport
    report = Report(metrics=[
        DataDriftPreset(),
        DataQualityPreset()
    ])
    
    report.run(reference_data=reference_data, current_data=production_data)
    
    # Sauvegarder le rapport
    report.save_html('../notebooks/evidently_drift_report.html')
    
    print("\n✅ Rapport Evidently généré: evidently_drift_report.html")
    print("📂 Ouvrez le fichier HTML dans votre navigateur pour voir le rapport complet.")
    
except ImportError:
    print("⚠️ Evidently n'est pas installé. Installez-le avec: pip install evidently")
except Exception as e:
    print(f"⚠️ Erreur lors de la génération du rapport Evidently: {e}")

## 7. Recommandations

In [ ]:
print("\n" + "=" * 80)
print("📋 RECOMMANDATIONS")
print("=" * 80)

# Compter les drifts
n_drifts = sum(drift_df['Drift'])

if n_drifts == 0:
    print("\n✅ STATUT: Aucun drift significatif détecté")
    print("\n💡 Le modèle est stable et peut continuer à fonctionner en production.")
    print("   Continuez à monitorer régulièrement les performances.")
    
elif n_drifts <= 2:
    print(f"\n⚠️ STATUT: Drift détecté sur {n_drifts} feature(s)")
    print("\n💡 Recommandations:")
    print("   1. Monitorer de près l'évolution des performances")
    print("   2. Collecter plus de données de production")
    print("   3. Planifier un re-entraînement dans les prochaines semaines")
    
else:
    print(f"\n🔴 STATUT: Drift significatif détecté sur {n_drifts} features")
    print("\n💡 Actions recommandées (URGENT):")
    print("   1. ⚠️ RE-ENTRAÎNER LE MODÈLE immédiatement avec des données récentes")
    print("   2. Analyser les causes du drift (changement de comportement client, saisonnalité, etc.)")
    print("   3. Augmenter la fréquence de monitoring")
    print("   4. Considérer un système de re-entraînement automatique")

print("\n📊 Features avec drift:")
for _, row in drift_df[drift_df['Drift']].iterrows():
    print(f"   - {row['Feature']}: p-value = {row['P-Value']:.4f}")

print("\n" + "=" * 80)

## 8. Export du rapport

In [ ]:
# Sauvegarder les résultats
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
drift_df.to_csv(f'../notebooks/drift_report_{timestamp}.csv', index=False)

print(f"\n💾 Rapport sauvegardé: drift_report_{timestamp}.csv")
print("\n✅ Analyse terminée!")